In [750]:
import os
import requests
import pandas as pd
import json
from dotenv import load_dotenv
import hvplot.pandas

%matplotlib inline

In [751]:
from nba_api.stats.static import players
from nba_api.stats.static import teams 
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import playergamelog
from nba_api.stats.library.parameters import NumberOfGames, SeasonAll, SeasonTypeAllStar, LeagueIDNullable

In [752]:
#getting player's game ID
#how to use function: player_id("Player's Name")
#for example: team_id('LeBron James')

player_dict = players.get_players()
teams = teams.get_teams()

def player_id(playername):
    player = [player for player in player_dict if player['full_name'] == playername][0]
    player_id = player['id']
    return player_id

#getting team's ID
#how to use function: player_id("Team's Name") 
#for example: team_id('Golden State Warriors')
def team_id(teamname):
    team_find = [x for x in teams if x['full_name'] == teamname][0]
    team_id_no = team_find['id']
    return team_id_no

#creating a function to get player's data from specific year
#type game_log(playerID, season's year)
#for example: game_log(2544,2018)

def game_log(playerid,season):
    player_gamelog = playergamelog.PlayerGameLog(player_id = playerid, season = season)
    df_games = player_gamelog.get_data_frames()
    return df_games[0]

#creating a function to get a player's lifetime career in basketball
#type all_season_log(playerID using earlier function)
def all_season_log(playerid):
    gamelog_bron_all = playergamelog.PlayerGameLog(player_id = playerid, season = SeasonAll.all)
    df_bron_games_all = gamelog_bron_all.get_data_frames()
    return df_bron_games_all[0]

#creating a function to get a team's stats in basketball
#type bkb_teamlog(Team's ID using earlier function)

def bkb_teamlog(teamid):
    team_games = leaguegamefinder.LeagueGameFinder(team_id_nullable = teamid).get_data_frames()[0]
    return team_games

#Summary: 
# 1) player_id('Player's Name')
# 2) team_id('Team's Name')
# 3) game_log(playerid,season)
# 4) all_season_log(playerid)
# 5) bkb_teamlog(teamid)

In [753]:
bron_id = player_id('LeBron James')
chris_id = player_id('Chris Paul')
james_id = player_id('James Harden')

print(f"Lebron ID      : {bron_id}")
print(f"Chris Paul ID  : {chris_id}")
print(f"James Harden ID: {james_id}")

a = team_id('Cleveland Cavaliers')
b = team_id('Los Angeles Lakers')
c = team_id('Miami Heat')

print(f"Cleveland Cavaliers Team ID :{a}")
print(f"Los Angeles Lakers Team ID  :{b}")
print(f"Miami Heat Team ID          :{c}")

#'Cleveland Cavaliers', 'Los Angeles Lakers','Miami Heat'

Lebron ID      : 2544
Chris Paul ID  : 101108
James Harden ID: 201935
Cleveland Cavaliers Team ID :1610612739
Los Angeles Lakers Team ID  :1610612747
Miami Heat Team ID          :1610612748


In [754]:
#To extract team's raw data from a specific date

def team_raw_data(start_date, end_date, teamid):
    team_raw_data = bkb_teamlog(teamid)
    range = (team_raw_data['GAME_DATE'] >= start_date) & (team_raw_data['GAME_DATE'] <= end_date)
    date_extract_data = team_raw_data.loc[range]
    return date_extract_data

In [755]:
#without Lebron

cavs_DF1 = team_raw_data('1999-02-02','2003-04-16',1610612739)
miami_DF = team_raw_data('2004-11-02','2009-04-15',1610612748)
cavs_DF2 = team_raw_data('2007-10-30','2013-04-17',1610612739)
la_lakers_before_lebron = team_raw_data('2014-10-28','2018-04-11',1610612747)

cavs_DF1_withlebron = team_raw_data('2066-10-31','2010-04-14',1610612739)
miami_DF_withlebron = team_raw_data('2010-10-26','2014-04-16',1610612748)
cavs_DF2_withlebron = team_raw_data('2014-10-28','2017-04-12',1610612739)
la_lakers_DF_withlebron = team_raw_data('2018-10-16','2022-04-04',1610612747)

In [756]:
#Extracting only "Years" from "GAME_DATE"
la_lakers_before_lebron['Year'] = pd.DatetimeIndex(la_lakers_before_lebron['GAME_DATE']).year
la_lakers_nolebron_DF = la_lakers_before_lebron

#Including only columns which we are analysising
a = la_lakers_nolebron_DF.loc[:,['GAME_DATE', 'Year', 'PTS','WL']]

lal_DF_nolebron = a.rename(columns={"PTS": "LAL_Mean_no_Lebron"})
lal_mean_DF = lal_DF_nolebron.groupby("Year").mean().reset_index()
lal_mean_DF

,Year,LAL_Mean_no_Lebron
0,2014,102.562500
1,2015,95.250000
2,2016,99.785714
3,2017,105.084211
4,2018,108.638298


In [757]:
years = lal_mean_DF['Year'].tolist()
years

[2014, 2015, 2016, 2017, 2018]

In [758]:
#Calculating standard deviation of each year's points
#Have to extract each year's data separately for std.function to work
#append each year's SD into a dataframe
team_SD = pd.DataFrame()
team_winrate = []
for year in years:
    lal_SD = lal_DF_nolebron.loc[lal_DF_nolebron['Year'] == year].std().drop("Year")
    a = list(lal_SD)
    team_SD = team_SD.append(a)
     
    b = la_lakers_nolebron_DF.loc[la_lakers_nolebron_DF['Year'] == year]
    lal_WR = b['WL'].to_list()
    winrate = lal_WR.count('W')/len(lal_WR)
    team_winrate.append(winrate)

C:\Users\yfkok\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  import sys


In [759]:
#creating winrate dataframe
WR_DF = pd.DataFrame(team_winrate) 
WR_DF['Year'] = years
WR_DF = WR_DF.set_index('Year').rename(columns={0: 'LAL_WR'})

#creating standard deviation
team_SD['Year'] = years
SD_DF = team_SD.set_index('Year').rename(columns={0: 'LAL_SD'})

#concatenate both mean & SD values on the same dataframe
lal_mean_DF = lal_mean_DF.set_index("Year")
LAL_main_DF = pd.concat([lal_mean_DF, SD_DF, WR_DF], axis='columns')

LAL_main_DF

,LAL_Mean_no_Lebron,LAL_SD,LAL_WR
Year,,,
2014,102.562500,10.162986,0.312500
2015,95.250000,12.509365,0.218750
2016,99.785714,12.610133,0.285714
2017,105.084211,12.288780,0.347368
2018,108.638298,11.664561,0.510638


In [760]:
#Extracting only "Years" from "GAME_DATE"
la_lakers_DF_withlebron['Year'] = pd.DatetimeIndex(la_lakers_DF_withlebron['GAME_DATE']).year
la_lakers_lebron_DF = la_lakers_DF_withlebron

#Including only columns which we are analysising
a = la_lakers_lebron_DF.loc[:,['GAME_DATE', 'Year', 'PTS','WL']]

lal_DF_lebron = a.rename(columns={"PTS": "LAL_Mean_Lebron"})
lal_mean_lebron_DF = lal_DF_lebron.groupby("Year").mean().reset_index()

In [761]:
years = lal_mean_lebron_DF['Year'].tolist()


In [762]:
#Calculating standard deviation of each year's points
#Have to extract each year's data separately for std.function to work
#append each year's SD into a dataframe
team_SD2 = pd.DataFrame()
team_winrate2 = []
for year in years:
    lal_SD = lal_DF_lebron.loc[lal_DF_lebron['Year'] == year].std().drop("Year")
    a = list(lal_SD)
    team_SD2 = team_SD2.append(a)
    
    a = lal_DF_lebron.loc[lal_DF_lebron['Year'] == year]
    lal_WR = a['WL'].to_list()
    winrate = lal_WR.count('W')/len(lal_WR)
    team_winrate2.append(winrate)

C:\Users\yfkok\anaconda3\envs\pyvizenv\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  import sys


In [763]:
#creating winrate dataframe
WR_DF2 = pd.DataFrame(team_winrate2) 
WR_DF2['Year'] = years
WR_DF2 = WR_DF2.set_index('Year').rename(columns={0: 'LAL_WR'})

#creating standard deviation
team_SD2['Year'] = years
SD_DF2 = team_SD2.set_index('Year').rename(columns={0: 'LAL_SD'})

#concatenate both mean & SD values on the same dataframe
lal_mean_lebron_DF = lal_mean_lebron_DF.set_index("Year")
LAL_main_with_Lebron_DF = pd.concat([lal_mean_lebron_DF, SD_DF2, WR_DF2], axis='columns')

LAL_main_with_Lebron_DF

,LAL_Mean_Lebron,LAL_SD,LAL_WR
Year,,,
2018,113.351351,10.799219,0.567568
2019,109.021739,13.832577,0.521739
2020,114.154930,11.789399,0.718310
2021,107.064516,13.295113,0.516129
2022,112.073171,10.320829,0.317073


In [764]:
LAL_main_DF

,LAL_Mean_no_Lebron,LAL_SD,LAL_WR
Year,,,
2014,102.562500,10.162986,0.312500
2015,95.250000,12.509365,0.218750
2016,99.785714,12.610133,0.285714
2017,105.084211,12.288780,0.347368
2018,108.638298,11.664561,0.510638
